In [1]:
import os

In [2]:
%pwd

'd:\\Weather_classification\\Weather-Image-Classification\\research'

In [ ]:
# os.chdir("D:/Weather_classification/Weather-Image-Classification")

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Weather_classification\\Weather-Image-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

#entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : str
    unzip_dir : Path

In [7]:
from Weather_Image_Classification.constants import *
from Weather_Image_Classification.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManger:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # print(self.config,type(self.config))
        
        # print(s)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from Weather_Image_Classification import  logger
from Weather_Image_Classification.utils.common import get_file_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded to following info \n{headers}")
        else:
            print(self.config.local_data_file)
            logger.info(f"{self.config.local_data_file} already exists with size of {get_file_size(path = self.config.local_data_file)}")

    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        print(self.config.local_data_file)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [11]:
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-26 14:27:37,433] || INFO || common || yaml file : config\config.yaml loaded successfully]
[2024-11-26 14:27:37,441] || INFO || common || yaml file : params.yaml loaded successfully]
[2024-11-26 14:27:37,442] || INFO || common || Directory artifacts already exists]
[2024-11-26 14:27:37,443] || INFO || common || Directory artifacts/data_ingestion already exists]
[2024-11-26 14:27:52,935] || INFO || 1162273902 || artifacts/data_ingestion/data.zip downloaded to following info 
Connection: close
Content-Length: 82641724
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "32a8dd551aebcae4ba5c8e1c59fac26f71fdc04d1d974b73c3264a86d22e003e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6D69:340805:7E397:9F345:67458D89
Accept-Ranges: bytes
Date: Tue, 26 Nov 2024 08:57:50 GMT
Via: 1.1 varnish
X-